# Aufgabe 10: 3-SAT Problem

Befehl env/Scripts/activate zum Starten der Virtuellen Umgebung


In [183]:
import numpy as np
rng = np.random.default_rng()

################    Parameter   ################

anzahlVariablen = 20
anzahlKlauseln = 20
maxLiteraleProKlausel = 3

maxGenerationen = 30
populationsGroesse = 100
anzahlSelektion = 20
anzahlMutation = 20
anzahlRekombination = 30 #muss eine Gerade Zahl sein
anzahlNeueIndividuen = 30

################################################

In [184]:
def createRandomKNF():
    Klauselmenge = []
    for i in range(anzahlKlauseln):
        numLiterale = rng.integers(low=1, high=(maxLiteraleProKlausel + 1))
        Klausel = []
        for k in range(numLiterale):
            literal = rng.integers(low = 1, high=(anzahlVariablen + 1))
            literal = -1*literal if rng.integers(low=0, high=2) == 1 else literal
            while(literal in Klausel or -1*literal in Klausel):
                literal = rng.integers(low = 1, high=(anzahlVariablen + 1))
                literal = -1*literal if rng.integers(low=0, high=2) == 1 else literal
            Klausel.append(literal)
        for k in range(maxLiteraleProKlausel - numLiterale):
            Klausel.append(0)
        Klauselmenge.append(Klausel)
    
    return Klauselmenge
    

#alt:
#for i in range(25):
    #numLiterale = rng.integers(low=1, high=4)
    #if numLiterale == 1:
        #Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), 0, 0])
    #if numLiterale == 2:
        #Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), 0])
    #if numLiterale == 3:
        #Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21))))])



In [185]:
def createWordKNF(klauselmenge):
    wordKlauselmenge = []
    for klausel in klauselmenge:
        wordKlausel =[]
        for i,variable in enumerate(klausel):
            if variable == 0:
                wordKlausel.append("f ∨ ")
            elif variable >0:
                number = int(variable)
                number = str(number)
                number = ''.join(chr(8320 + int(digit)) for digit in number)
                wordKlausel.append("x" + number + " ∨ ")
            elif variable <0:
                number = abs(int(variable))
                number = str(number)
                number = ''.join(chr(8320 + int(digit)) for digit in number)
                wordKlausel.append("¬x" + number + " ∨ ")
        wordKlausel[-1] = wordKlausel[-1].replace(" ∨ ", '')
            
        wordKlauselmenge.append(wordKlausel)
    return(wordKlauselmenge)

def printWordKNF(wordKlauselmenge):
    for i, Klausel in enumerate(wordKlauselmenge):
        print("(", end="")
        for Variable in Klausel:
            print(Variable, end="")
        if i == (len(wordKlauselmenge) - 1):
            print(")")
        else: 
            print(") ∧ " ,end="")
        
    

In [186]:
def InitPopulation():
    initPopulation = rng.integers(size=(populationsGroesse, (anzahlVariablen + 1)), low=0, high=2)
    for individuum in initPopulation:
        individuum[0] = 1

    return initPopulation

In [187]:
def sortPopulation(population, knf):
    bewertung = {}
    for i, individuum in enumerate(population):
        erfüllteKlauseln = 0
        for klausel in knf:
            for variable in klausel:
                if variable > 0 and individuum[variable] == 1:
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
                elif variable < 0 and individuum[-1 * variable] == 0:
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
        bewertung[i] = erfüllteKlauseln

    sortierteIndividuen = dict(sorted(bewertung.items(), key=lambda item: item[1]))

    return sortierteIndividuen

In [188]:
def Selektion(sortedPopulation, population):
    selected = []
    np.array(selected)
    for i in range(anzahlSelektion):
        individuum = population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i]]
        #print(individuum)
        selected.extend(individuum)
        
        #selected = np.concatenate((selected,individuum ), axis=0)
        #selected[99-i] = [individuum]
        #selectet = selected + individuum
        #selected
    selected = np.reshape(selected, (anzahlSelektion, (anzahlVariablen + 1)))
    
    return selected

In [189]:
def Mutation(sortedPopulation, population):
    mutatedPopulation = []
    for i in range(anzahlMutation):
        individuum = population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i] ]
        mutationSpot = rng.integers(low=1, high=(anzahlVariablen + 1))
        individuum[mutationSpot] = 1 if individuum[mutationSpot] == 0 else 0
        mutatedPopulation.append(individuum)

    return mutatedPopulation
        

In [190]:
def Rekombination(sortedPopulation, population):
    recombinedPopulation = []
    for i in range(int(anzahlRekombination / 2)):
        individuum1 = np.array_split(population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i]], 2)
        individuum2 = np.array_split(population[list(sortedPopulation.keys())[rng.integers(low=1, high=21)]], 2)

        new1 = np.concatenate((individuum1[0], individuum2[1]))
        new2 = np.concatenate((individuum2[0], individuum1[1]))

        recombinedPopulation.append(new1)
        recombinedPopulation.append(new2)
        
    return recombinedPopulation

In [191]:
def createNewRandom():
    newRandoms = rng.integers(size=(anzahlNeueIndividuen, (anzahlVariablen+1)), low=0, high=2)
    for individuum in newRandoms:
        individuum[0] = 1
        
    return newRandoms

In [192]:
def Evolution(knf, iterations, startPopulation):
    population = startPopulation
    for i in range(iterations):
        sortedPopulation = sortPopulation(population, knf)

        #print(list(sortedPopulation.values())[populationsGroesse - 1])
        if(list(sortedPopulation.values())[populationsGroesse - 1] == anzahlKlauseln):
            print(f'Erfüllende Belegung gefunden in der {i}. Generation')
            return population[list(sortedPopulation.keys())[(populationsGroesse - 1)]]

        selektion = Selektion(sortedPopulation, population)
        mutation = Mutation(sortedPopulation, population)
        rekombination = Rekombination(sortedPopulation, population)
        newRandoms = createNewRandom()

        newPopulation = np.concatenate((selektion, newRandoms,mutation,rekombination ),  axis=0)

        population = newPopulation
    print(f'Nach {maxGenerationen} Generationen wurde keine Erfüllende Belegung gefunden')
    return 'Kontradiktion'

In [193]:
def print_marked_word_knf(knf, belegung):
    GREEN = '\033[92m'
    RESET = '\033[0m'
    RED = '\033[91m'

    ausgabe = ''
    for i, klausel in enumerate(knf):
        ausgabe += "("
        for literal in klausel:
            if literal == 0:
                #ausgabe += 'f ∨ '
                continue
            else:
                variable = abs(literal)
                literal_string = ('x' if literal > 0 else '¬x') + str(variable)
                if (literal > 0 and belegung[variable]) or (literal < 0 and not belegung[variable]):
                    ausgabe += GREEN + literal_string + RESET + ' ∨ '
                else:
                    ausgabe += literal_string + ' ∨ '

        ausgabe = ausgabe.rstrip(' ∨ ')
        ausgabe += ")" if i == len(knf) - 1 else ") ∧ "

    print(ausgabe)


In [194]:
#benutzereingabe = 4
#while benutzereingabe <0 or benutzereingabe >3:
    #benutzereingabe = int(input("Bitte Modus wählen \n  1 = zufällige \n 2 = Eingabe einer Kontradiktion \n "))


#if benutzereingabe ==1:
    #knf = createRandomKNF()
#elif benutzereingabe ==2:
    #Kontradiktionserstellung
    #print(benutzereingabe)
#wordKNF = generateWordKNF(knf)
#printWordKNF(wordKNF)

In [195]:
knf = createRandomKNF()
wordKNF = createWordKNF(knf)
printWordKNF(wordKNF)

startPopulation = InitPopulation()
belegung = Evolution(knf, maxGenerationen, startPopulation)
print(belegung)

print_marked_word_knf(knf, belegung)

(x₁₀ ∨ x₁₇ ∨ ¬x₂₀) ∧ (x₁₃ ∨ x₁₉ ∨ f) ∧ (¬x₁₀ ∨ ¬x₅ ∨ f) ∧ (x₁₉ ∨ x₁₆ ∨ x₁₃) ∧ (¬x₁₁ ∨ f ∨ f) ∧ (¬x₉ ∨ f ∨ f) ∧ (¬x₇ ∨ x₁₉ ∨ x₃) ∧ (¬x₇ ∨ ¬x₁₃ ∨ f) ∧ (x₉ ∨ x₇ ∨ ¬x₁₉) ∧ (¬x₁₉ ∨ f ∨ f) ∧ (¬x₁₂ ∨ ¬x₁₈ ∨ f) ∧ (¬x₁₀ ∨ ¬x₁₂ ∨ f) ∧ (x₁₂ ∨ f ∨ f) ∧ (¬x₄ ∨ x₁₁ ∨ x₁₈) ∧ (¬x₁₁ ∨ ¬x₂ ∨ f) ∧ (x₇ ∨ ¬x₁₇ ∨ f) ∧ (x₁₇ ∨ ¬x₃ ∨ ¬x₁₁) ∧ (¬x₉ ∨ x₁₇ ∨ x₁₉) ∧ (x₁₇ ∨ ¬x₇ ∨ ¬x₆) ∧ (x₁₂ ∨ ¬x₇ ∨ f)
Erfüllende Belegung gefunden in der 19. Generation
[1 1 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
(x10 ∨ x17 ∨ ¬x20) ∧ (x13 ∨ x19) ∧ (¬x10 ∨ ¬x5) ∧ (x19 ∨ x16 ∨ x13) ∧ (¬x11) ∧ (¬x9) ∧ (¬x7 ∨ x19 ∨ x3) ∧ (¬x7 ∨ ¬x13) ∧ (x9 ∨ x7 ∨ ¬x19) ∧ (¬x19) ∧ (¬x12 ∨ ¬x18) ∧ (¬x10 ∨ ¬x12) ∧ (x12) ∧ (¬x4 ∨ x11 ∨ x18) ∧ (¬x11 ∨ ¬x2) ∧ (x7 ∨ ¬x17) ∧ (x17 ∨ ¬x3 ∨ ¬x11) ∧ (¬x9 ∨ x17 ∨ x19) ∧ (x17 ∨ ¬x7 ∨ ¬x6) ∧ (x12 ∨ ¬x7)
